In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from attacks import GM
from classifiers import MNIST_CNN
from utils import save, unpickle, pixel_range, preprocess, hist_kde
from vae_gans import MNIST_VAE_GAN

fmnist = input_data.read_data_sets("FMNIST_data/")

data_train = (np.reshape(fmnist.train.images, [-1, 28, 28, 1]) * 2.0 - 1, fmnist.train.labels)
data_test = (np.reshape(fmnist.test.images, [-1, 28, 28, 1]) * 2.0 - 1, fmnist.test.labels)

label_map = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


## Loss Gradient Visualization

In [ ]:
tf.reset_default_graph()

sess = tf.InteractiveSession()

DNN = MNIST_CNN('tf_logs/exp2/standard/', activation=tf.nn.relu)
DNN.load(sess)

loss = -tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.stop_gradient(tf.one_hot(DNN.yi, depth=DNN.n_classes)),
                                                   logits=DNN.logits)

loss_grad = tf.gradients(loss, DNN.X)[0]

r1 = preprocess(sess.run(loss_grad, feed_dict={DNN.X: data_test[0][:100]}), q1=0.5, q2=99.5)

sess.close()

tf.reset_default_graph()

sess = tf.InteractiveSession()

DNN = MNIST_CNN('tf_logs/exp2/adv/0.2_0.02_40_inf_xent/', activation=tf.nn.relu)
DNN.load(sess)

loss = -tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.stop_gradient(tf.one_hot(DNN.yi, depth=DNN.n_classes)),
                                                   logits=DNN.logits)

loss_grad = tf.gradients(loss, DNN.X)[0]

r2 = preprocess(sess.run(loss_grad, feed_dict={DNN.X: data_test[0][:100]}), q1=0.5, q2=99.5)

sess.close()

tf.reset_default_graph()

sess = tf.InteractiveSession()

DNN = MNIST_CNN('tf_logs/exp2/adv/4_2_40_2_xent/', activation=tf.nn.relu)
DNN.load(sess)

loss = -tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.stop_gradient(tf.one_hot(DNN.yi, depth=DNN.n_classes)),
                                                   logits=DNN.logits)

loss_grad = tf.gradients(loss, DNN.X)[0]

r3 = preprocess(sess.run(loss_grad, feed_dict={DNN.X: data_test[0][:100]}), q1=0.5, q2=99.5)

sess.close()


In [ ]:
inds = [1, 3, 7]

plt.figure(figsize=(9, 12))

for i, ind in enumerate(inds):

    plt.subplot(4, 3, i + 1)
    plt.imshow(data_test[0][ind].reshape(28, 28), cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title(label_map[data_test[1][ind]], fontsize=25)
    if i == 0: plt.ylabel('Original', fontsize=25)

    plt.subplot(4, 3, 4 + i)
    v, _ = pixel_range(r1[ind])
    plt.imshow(r1[ind].reshape(28, 28), vmin=v[0], vmax=v[1], cmap='bwr')
    plt.xticks([])
    plt.yticks([])
    if i == 0: plt.ylabel('Standard', fontsize=25)

    plt.subplot(4, 3, 7 + i)
    v, _ = pixel_range(r2[ind])
    plt.imshow(r2[ind].reshape(28, 28), vmin=v[0], vmax=v[1], cmap='bwr')
    plt.xticks([])
    plt.yticks([])
    if i == 0: plt.ylabel('L-Inf Trained', fontsize=25)

    plt.subplot(4, 3, 10 + i)
    v, _ = pixel_range(r3[ind])
    plt.imshow(r3[ind].reshape(28, 28), vmin=v[0], vmax=v[1], cmap='bwr')
    plt.xticks([])
    plt.yticks([])
    if i == 0: plt.ylabel('L2 Trained', fontsize=25)

plt.tight_layout()
plt.subplots_adjust(wspace=0.04, hspace=0.04)

plt.show()

plt.close()


## Distance to Projection

In [ ]:
def evaluate(tf_logs, attack_params):
    datasets = []
    reconsts = []
    suc_inds = []
    l2_dists = []

    print('Generating Adversarial Attacks')

    for name, tf_log in tf_logs:

        if not tf_log:
            datasets.append((name, data_test[0]))
            continue

        tf.reset_default_graph()

        sess = tf.InteractiveSession()

        DNN = MNIST_CNN(tf_log, activation=tf.nn.relu)
        DNN.load(sess)

        gm = GM(DNN, **attack_params)
        datasets.append((name, gm.attack(sess, data_test, batch_size=500)))

        sess.close()

    print('Generating Projections')

    tf.reset_default_graph()

    sess = tf.InteractiveSession()

    vae_gan = MNIST_VAE_GAN('tf_logs/exp2/vae-gan/')
    vae_gan.load(sess)

    for name, dataset in datasets:
        reconsts.append((name, vae_gan.reconstruct(sess, (dataset, 0))))

    sess.close()

    print('Removing Unsuccessful Attacks')

    tf.reset_default_graph()

    sess = tf.InteractiveSession()

    DNN = MNIST_CNN('tf_logs/exp2/standard/', activation=tf.nn.relu)
    DNN.load(sess)

    org_ys = DNN.inference(sess, data_test, batch_size=500)

    for name, dataset in datasets:

        adv_ys = DNN.inference(sess, (dataset, 0), batch_size=500)

        if name == 'Test Data':
            suc_inds.append(org_ys == adv_ys)
        else:
            suc_inds.append(org_ys != adv_ys)

    sess.close()

    print('Calculating L2 Distances')

    for i, ((name, dataset), (_, reconst)) in enumerate(zip(datasets, reconsts)):
        l2_dist = np.sqrt(np.sum(np.square(dataset[suc_inds[i]] - reconst[suc_inds[i]]), axis=(1, 2, 3)))
        l2_dists.append((name, l2_dist))

    return datasets, reconsts, l2_dists


In [ ]:
tf_logs1 = [('Test Data', None),
            ('Standard', 'tf_logs/exp2/standard/'),
            ('L2 1 XEnt', 'tf_logs/exp2/adv/1_2_40_2_cw/'),
            ('L2 2 XEnt', 'tf_logs/exp2/adv/2_2_40_2_cw/'),
            ('L2 4 XEnt', 'tf_logs/exp2/adv/4_2_40_2_cw/')
            ]

tf_logs2 = [('Test Data', None),
            ('Standard', 'tf_logs/exp2/standard/'),
            ('L2 1 CW', 'tf_logs/exp2/adv/1_2_40_2_cw/'),
            ('L2 2 CW', 'tf_logs/exp2/adv/2_2_40_2_cw/'),
            ('L2 4 CW', 'tf_logs/exp2/adv/4_2_40_2_cw/')
            ]

tf_logs3 = [('Test Data', None),
            ('Standard', 'tf_logs/exp2/standard/'),
            ('L-Inf 0.05 XEnt', 'tf_logs/exp2/adv/0.05_0.02_40_inf_xent/'),
            ('L-Inf 0.1 XEnt', 'tf_logs/exp2/adv/0.1_0.02_40_inf_xent/'),
            ('L-Inf 0.2 XEnt', 'tf_logs/exp2/adv/0.2_0.02_40_inf_xent/'),
            ]

tf_logs4 = [('Test Data', None),
            ('Standard', 'tf_logs/exp2/standard/'),
            ('L-Inf 0.05 CW', 'tf_logs/exp2/adv/0.05_0.02_40_inf_cw/'),
            ('L-inf 0.1 CW', 'tf_logs/exp2/adv/0.1_0.02_40_inf_cw/'),
            ('L-inf 0.2 CW', 'tf_logs/exp2/adv/0.2_0.02_40_inf_cw/'),
            ]

attack_params = {'eps': 12, 'step_size': 2, 'n_steps': 40, 'norm': '2', 'loss_type': 'xent'}
# attack_params = {'eps': 12, 'step_size': 2, 'n_steps': 40, 'norm': '2', 'loss_type': 'cw'}

ress = []

for tf_logs in [tf_logs1, tf_logs2, tf_logs3, tf_logs4]:
    ress.append(evaluate(tf_logs, attack_params))

save(ress, 'results/gradient/exp2/results_{}.pickle'.format(attack_params['loss_type']))


In [ ]:
ress = unpickle('results/gradient/exp2/results_xent.pickle')

plt.figure(figsize=(16, 3))

for i, res in enumerate(ress):

    plt.subplot(1, 4, i + 1)

    for j, (name, l2_dist) in enumerate(res[2]):

        if name == 'Test Data':
            xmin = np.min(l2_dist)
            xmax = np.max(l2_dist)

        xs, ys = hist_kde((l2_dist - xmin) / (xmax - xmin), [0.0, 1.0])
        plt.plot(xs, ys, label=name)

    plt.xlim([0, 1])
    plt.ylim([0, 8])
    plt.xlabel('L2 Distance to Manifold', fontsize=12)
    plt.legend(loc='upper left', fontsize=10)

plt.tight_layout()

plt.show()
plt.close()


In [ ]:
ress = unpickle('results/gradient/exp2/results_cw.pickle')

plt.figure(figsize=(16, 3))

for i, res in enumerate(ress):

    plt.subplot(1, 4, i + 1)

    for j, (name, l2_dist) in enumerate(res[2]):

        if name == 'Test Data':
            xmin = np.min(l2_dist)
            xmax = np.max(l2_dist)

        xs, ys = hist_kde((l2_dist - xmin) / (xmax - xmin), [0.0, 1.0])
        plt.plot(xs, ys, label=name)

    plt.xlim([0, 1])
    plt.ylim([0, 8])
    plt.xlabel('L2 Distance to Manifold', fontsize=12)
    plt.legend(loc='upper left', fontsize=10)

plt.tight_layout()

plt.show()
plt.close()


## Adversarial Attack Visualization

In [ ]:
# ress = unpickle('results/gradient/exp2/results_xent.pickle')
# ress = unpickle('results/gradient/exp2/results_cw.pickle')

i = 0

plt.figure(figsize=(3 * len(ress[0][0]), 6))

for j, (name, x) in enumerate(ress[0][0]):

    plt.subplot(2, len(ress[0][0]), j + 1)
    plt.imshow(x[i].reshape(28, 28), cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.title('Original' if name == 'Test Data' else name, fontsize=20, pad=10)

    if j == 0:
        plt.ylabel('FMNIST', fontsize=20, labelpad=10)

plt.tight_layout()
plt.subplots_adjust(wspace=-0.15, hspace=0.03)

plt.show()

plt.close()
